In [1]:
%%bash

# Dependencies
pip install docx2txt
pip install gensim
pip install keras
pip install nltk
pip install -U scikit-learn
pip install python-docx
pip install tensorflow
pip install pandas
pip install openpyxl
pip install nltk
pip install spacy
pip install csv
python3 -m spacy download en_core_web_sm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
2023-08-31 01:55:22.448104: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 01:55:22.566729: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 01:55:22.567606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 01:55:23.458026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-31 01:55:24.986507: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import pandas as pd
import csv
import math
import pprint

import spacy
# Use if en_core_web_sm not installable via python3 in terminal:
# spacy.cli.download("en_core_web_sm")

2023-08-31 01:55:30.781886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 01:55:30.821605: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-31 01:55:30.822759: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 01:55:31.496397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-31 01:55:32.238059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-31 01:55:32.238480: W tensorflow/core/common_r

In [3]:
lemmatizer = WordNetLemmatizer()

def verb_classifier(verbs_file_path): # verbs_file_path - string value that contains the file path

    xlsx = pd.ExcelFile(verbs_file_path, engine='openpyxl') 

    sheet_names = xlsx.sheet_names  # Get a list of sheet names

    # Create an empty dictionary to store DataFrames for each sheet
    dfs = {}

    for sheet_name in sheet_names:
        df = xlsx.parse(sheet_name)  # For XLSX files
        
        # Store the DataFrame in the dictionary
        dfs[sheet_name] = df

    domain_levels = pd.concat(dfs)
    duplicate_checklist = []

    for i in range(domain_levels.shape[0]):
        for j in range(domain_levels.shape[1]):
            cell_value = domain_levels.iloc[i, j]
            if not pd.isna(cell_value):
                cell_value_lower = cell_value.lower()
                verb = lemmatizer.lemmatize(cell_value_lower)
                domain_levels.iloc[i, j] = verb
                if cell_value_lower not in duplicate_checklist:
                    duplicate_checklist.append(cell_value_lower)
                else:
                    domain_levels.iloc[i, j] = float('nan')

    return domain_levels


# Paths
solo_file_path = 'SOLO.xlsx'
bloom_cognitive_file_path = 'Bloom_cognitive.xlsx'
bloom_psychomotor_file_path = 'Bloom_psychomotor.xlsx'
bloom_affective_file_path = 'Bloom_affective.xlsx'

# Verbs
mapped_verbs = {
    "Cognitive": verb_classifier(bloom_cognitive_file_path),
    "Affective": verb_classifier(bloom_affective_file_path),
    "Psychomotor": verb_classifier(bloom_psychomotor_file_path),
    "SOLO": verb_classifier(solo_file_path)
}


In [4]:
#### Check the dataframe for duplicates

# verbs_df - The dataframe that consists of the classified verbs
# domain_name - string name of the domain. (SOLO, Blooms Cognitive, Blooms Affective or Blooms Psychomotor)
def check_duplicates(verbs_df, domain_name): 

    ## Checking duplicates for SOLO
    list_of_lists = verbs_df.values.tolist()
    merged_list = [item for sublist in list_of_lists for item in sublist]

    # Remove nan
    cleaned_list = list(filter(lambda x: not pd.isna(x), merged_list))

    # Check for duplicate
    if len(cleaned_list) != len(set(cleaned_list)):
        print("Duplicates Found in " + domain_name)
    else:
        print("No duplicates exist in " + domain_name)
    
    list_of_lists.clear()
    merged_list.clear()
    cleaned_list.clear()


# Check for duplicates
for taxonomy_key, taxonomy_item in mapped_verbs.items():
    check_duplicates(taxonomy_item, taxonomy_key)


No duplicates exist in Cognitive
Duplicates Found in Affective
No duplicates exist in Psychomotor
No duplicates exist in SOLO


In [5]:
## This code loads the vector file into the word_vectors variable
## Download the vector file from https://fasttext.cc/docs/en/english-vectors.html (first file on the website), unzip the file and store in your local development folder
## Note: This piece of code may take upto an hour or two to run depending on your pc specs.
## My i5 8th gen with 8gig ram took 58mins to run.

# Path to the downloaded .vec file
path_to_vectors = 'wiki-news-300d-1M.vec'
# path_to_vectors = 'wiki.en.vec'
# Load the word vectors
word_vectors = KeyedVectors.load_word2vec_format(path_to_vectors)

# Find similar words
similar_words = word_vectors.most_similar('cat')

# Calculate word similarity
similarity = word_vectors.similarity('cat', 'dog')

# Perform vector arithmetic
result = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']

In [6]:
## Function to identify verbs in a sentence
def identify_verbs(sentence):
    # Load the English language model in spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # Process the sentence using spaCy
    doc = nlp(sentence)
    
    # Extract the verbs from the processed sentence
    verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    
    return verbs

In [7]:
def extract_columns(csv_file, columns):
    extracted_data = {}
    
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the headers
        
        # Check if all specified columns exist in the CSV file
        for column in columns:
            if column not in headers:
                raise ValueError(f"Column '{column}' not found in the CSV file.")
        
        # Initialize separate arrays for each column
        for column in columns:
            extracted_data[column] = []
        
        # Extract data from specified columns
        for row in reader:
            for column in columns:
                column_index = headers.index(column)
                extracted_data[column].append(row[column_index])
    
    return extracted_data

# Example usage
csv_file = 'Learning outcomes manual mapping - Mappings.csv'
columns_to_extract = ['Learning outcomes', 'Final Bloom Level', 'Final SOLO Level']
extracted_data = extract_columns(csv_file, columns_to_extract)
sentences = extracted_data['Learning outcomes']
final_bloom_level = extracted_data['Final Bloom Level']
final_solo_level = extracted_data['Final SOLO Level']
final_levels = {
    "Cognitive": final_bloom_level,
    "SOLO": final_solo_level
}



In [8]:
## Main piece of code that performs the mapping - Approach 2 - 12% accuracy

# wiki word vectors no uppercase
# TODO: Modify this section of the code to use the bloom level verbs from Arragon's spreadsheet, will also need to modify
# Ideas for improving accuracy
#### Reduce the number of verbs.
#### Take more learning outcomes from the monash handbook website(need big dataset for this part) and identify verbs that are appearing multiple times 
#### The nummber of times that it appears could be set to a certain number ex: 5. 
#### If the verb doesnt appear atleast 5 times, we could remove the verb from our list of predefined verbs which will result in a shorter verb list

output_bloom_levels = []

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))

# Array of all the PLOs and ULOs (We can couple them together as we're trying to identify Bloom/Solo level here)
lo_sentence_array = []
bloom_levels_str = [
    "Remembering",
    "Understanding",
    "Applying",
    "Analysing",
    "Evaluating",
    "Creating",
]

# TODO: train CLO classification with all data instead of just one course.
for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    cleaned_tokens = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token.lower() not in stop_words
    ]
    lo_sentence_array.append(cleaned_tokens)

# build the vocabulary and train the model
# IMPORTANT, N0TE THAT sg=1 flag specifies Word2Vec to use the Skip Gram Model as designated by the LSTM paper.
model = Word2Vec(
    sentences=lo_sentence_array, vector_size=100, window=5, min_count=1, workers=4, sg=1
)

# train the model with the course's ULOs and PLOs.
model.train([tokens], total_examples=len([tokens]), epochs=10)

[nltk_data] Downloading package punkt to /home/arragon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arragon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/arragon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


(38, 60)

In [9]:
def bloom_mapping(sentences, final_levels):
    passed_mappings = {
            "Cognitive": 0,
            "Affective": 0,
            "Psychomotor": 0,
            "SOLO": 0
        }
    failed_mappings = {
            "Cognitive": 0,
            "Affective": 0,
            "Psychomotor": 0,
            "SOLO": 0
        }
    total_mappings = len(sentences) * 2
    failed_cases = []

    for i in range(len(sentences)): # Iterates over the LOs
        identified_verbs = identify_verbs(sentences[i])

        score_list = {
            "Cognitive": {
                "Remembering": 0,
                "Understanding": 0,
                "Applying": 0,
                "Analysing": 0,
                "Evaluating": 0,
                "Creating": 0
            },
            "Affective": {
                "Receiving": 0,
                "Responding": 0,
                "Valuing": 0,
                "Organisation": 0,
                "Characterisation": 0
            },
            "Psychomotor": {
                "Perception": 0,
                "Set": 0,
                "Guided Response": 0,
                "Mechanism": 0,
                "Complex Overt Response": 0,
                "Adaptation": 0,
                "Origination": 0    
            },
            "SOLO": {
                "Prestructural": 0,
                "Unistructural": 0,
                "Multistructural": 0,
                "Relational": 0,
                "Extended Abstract": 0
            }
        }

        for taxonomy_key, taxonomy_item in mapped_verbs.items():
            for j in range(taxonomy_item.shape[0]): # Level
                for k in range(taxonomy_item.shape[1]): # Verb
                    verb = taxonomy_item.iloc[j, k]
                    similarity_score = 0
                    for l in range(len(identified_verbs)):
                        try:    # Currently some of the 'verbs' identified are phrases rather than words and it was throwing errors so this is a temp solution 
                            similarity_score += word_vectors.similarity(identified_verbs[l], verb)
                        except:
                            pass
                    score_list[taxonomy_key][taxonomy_item.columns[k]] += similarity_score

        # Identify level based on similarity
        max_score = {
            "Cognitive": { "Level": None, "Score": 0 },
            "Affective": { "Level": None, "Score": 0 },
            "Psychomotor": { "Level": None, "Score": 0 },
            "SOLO": { "Level": None, "Score": 0 }
        }
        for t_key, t_item in score_list.items():
            for l in t_item:
                if max_score[t_key]["Score"] < score_list[t_key][l]:
                    max_score[t_key] = { "Level": l, "Score": score_list[t_key][l] }

        for t_key, t_item in final_levels.items():
            if final_levels[t_key][i] == None: continue
            
            if max_score[t_key]["Level"] != None and final_levels[t_key][i].lower() == max_score[t_key]["Level"].lower():
                passed_mappings[t_key] += 1
            else:
                failed_mappings[t_key] += 1
                sentence_data = {
                    "Domain": t_key,
                    "manually identified level": max_score[t_key]["Level"],
                    "automatically identified level": final_levels[t_key][i],
                    "verbs identified": identified_verbs,
                    "sentence": sentences[i],
                }
                failed_cases.append(sentence_data)

    for case in failed_cases:
        print(case)

    total_passed = 0
    for taxonomy, passed in passed_mappings.items():
        total_passed += passed
    total_mapping_percentage = math.ceil((total_passed/total_mappings)*100)
    print("Total percentage of mappings passed: ", total_mapping_percentage, "%")
    pass


### Todos
# Find a way to use skipgrams
# This method only works for blooms since this paper is only based on blooms mapping

bloom_mapping(sentences, final_levels)


{'Domain': 'SOLO', 'manually identified level': 'Relational', 'automatically identified level': 'Multistructural', 'verbs identified': ['propose'], 'sentence': 'Discern fundamental chemical, materials, mechanical and environmental engineering knowledge, principles and concepts to propose solutions to a humanitarian engineering problem.'}
{'Domain': 'Cognitive', 'manually identified level': 'Understanding', 'automatically identified level': 'Analysing', 'verbs identified': ['identify', 'provide', 'use'], 'sentence': 'Identify design requirements from a provided brief and analyse potential solutions using first principles of mathematics and natural and engineering sciences.'}
{'Domain': 'Cognitive', 'manually identified level': 'Understanding', 'automatically identified level': 'Remembering', 'verbs identified': ['identify', 'include'], 'sentence': 'Identify societal, health, safety, legal and cultural issues relevant to your project including the Indigenous context, and your consequent 